In [14]:
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

In [15]:
datasets, info = tfds.load(name='mnist', with_info = True, as_supervised = True)
mnist_train, mnist_test = datasets['train'], datasets['test']

Shuffling and writing examples to C:\/Users/June/tensorflow_datasets/mnist/3.0.1.incompleteS20W8P/mnist-train.tfrecord
Shuffling and writing examples to C:\/Users/June/tensorflow_datasets/mnist/3.0.1.incompleteS20W8P/mnist-test.tfrecord
Dataset mnist downloaded and prepared to C:\Users\June\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [39]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [40]:
print('Num devices: {}'.format(strategy.num_replicas_in_sync))

Num devices: 1


In [19]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [20]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    
    return image, label

In [21]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [22]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation = 'relu', input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics = ['accuracy'])

In [23]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpo_{epoch}')

In [24]:
def decay(epoch):
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

In [25]:
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nEpoch {} Learning rate: {}'.format(epoch + 1, model.optimizer.lr.numpy()))

In [26]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only = True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [27]:
model.fit(train_dataset, epochs = 12, callbacks = callbacks)

Epoch 1/12
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

937/938 [============================>.] - ETA: 0s - loss: 0.2154 - accuracy: 0.9378
Epoch 1 Learning rate: 0.0010000000474974513
938/938 [==============================] - 4s 4ms/step - loss: 0.2154 - accuracy: 0.9378
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Epoch 2/12
927/938 [============================>.] - ETA: 0s - loss: 0.0668 - accuracy: 0.9807
Epoch 2 Learning rate: 0.0010000000474974513
938/938 [==============================] - 3s 3ms/step - loss: 0.0666 - accuracy: 0.9808
Epoch 3/12
923/938 [============================>.] - ETA: 0s - loss: 0.0464 - accuracy: 0.9861
Epoch 3 Learning rate: 0.0010000000474974513
938/938 [==============================] - 3s 3ms/step - loss: 0.0464 - accuracy: 0.9860
Epoch 4/12
934/938 [============================>.] - ETA: 0s - loss: 0.0259 - accuracy: 0.9928
Epoch 4 Learning rate: 9.999999747378752e-05
938/938 [==============================] - 3s 3ms/step - loss: 0.0259 - accuracy: 0.9929
Epoch 5/12
938/938 [==============================] - ETA: 0s - loss: 0.0225 - accuracy: 0.9940
Epoch 5 Learning rate: 9.999999747378752e-05
938/938 [==============================] - 3s 3ms/step - loss: 0.0225 - accuracy: 0.9940
Epoch 6/12
920/938 [============================>.] - ETA: 0s - loss: 0.0209 - a

In [28]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
eval_loss, eval_acc = model.evaluate(eval_dataset)
print('Loss: {}, Accuracy: {}'.format(eval_loss, eval_acc))

  1/157 [..............................] - ETA: 0s - loss: 0.0884 - accuracy: 0.9688WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0000s vs `on_test_batch_end` time: 0.0104s). Check your callbacks.


157/157 [==============================] - 1s 4ms/step - loss: 0.0381 - accuracy: 0.9872
Loss: 0.03809528052806854, Accuracy: 0.9872000217437744


In [29]:
path = 'saved_model/'

In [32]:
model.save(path)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [35]:
unreplicated_model = tf.keras.models.load_model(path)
unreplicated_model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)
print("Loss: {}, Accuracy: {}".format(eval_loss, eval_acc))

157/157 [==============================] - 0s 2ms/step - loss: 0.0381 - accuracy: 0.9872
Loss: 0.03809528052806854, Accuracy: 0.9872000217437744


In [37]:
with strategy.scope():
    replicated_model = tf.keras.models.load_model(path)
    replicated_model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer = tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    
    eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
    print("Loss: {}, Accuracy: {}".format(eval_loss, eval_acc))

157/157 [==============================] - 0s 2ms/step - loss: 0.0381 - accuracy: 0.9872
Loss: 0.03809528052806854, Accuracy: 0.9872000217437744
